## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Mataura,-46.1927,168.8643,57.07,77,100,6.04,NZ,2021-03-14 22:39:50,overcast clouds
1,1,Castro,-24.7911,-50.0119,65.68,83,4,1.52,BR,2021-03-14 22:36:49,clear sky
2,2,Vigrestad,58.5667,5.7000,39.99,95,55,3.00,NO,2021-03-14 22:44:13,broken clouds
3,3,Kapaa,22.0752,-159.3190,73.40,78,75,23.02,US,2021-03-14 22:36:14,light rain
4,4,Bundaberg,-24.8500,152.3500,84.99,73,71,5.75,AU,2021-03-14 22:44:14,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 115


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
4,4,Bundaberg,-24.8500,152.3500,84.99,73,71,5.75,AU,2021-03-14 22:44:14,broken clouds
6,6,Hilo,19.7297,-155.0900,75.99,83,90,13.80,US,2021-03-14 22:43:00,moderate rain
11,11,Namatanai,-3.6667,152.4333,84.00,66,67,4.12,PG,2021-03-14 22:37:19,broken clouds
13,13,Hithadhoo,-0.6000,73.0833,84.09,64,97,7.23,MV,2021-03-14 22:36:17,overcast clouds
19,19,Gisborne,-38.6533,178.0042,75.00,56,6,4.00,NZ,2021-03-14 22:44:17,clear sky
23,23,Georgetown,5.4112,100.3354,78.80,83,20,5.75,MY,2021-03-14 22:36:03,few clouds
24,24,Pilar,-9.5972,-35.9567,78.80,69,0,10.36,BR,2021-03-14 22:44:19,clear sky
27,27,Saint-Philippe,-21.3585,55.7679,78.80,73,20,16.11,RE,2021-03-14 22:36:36,few clouds
29,29,Rikitea,-23.1203,-134.9692,80.87,76,18,13.15,PF,2021-03-14 22:36:21,moderate rain
33,33,Cabo San Lucas,22.8909,-109.9124,75.00,40,20,8.05,MX,2021-03-14 22:39:35,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                156
City                   156
Lat                    156
Lng                    156
Max Temp               156
Humidity               156
Cloudiness             156
Wind Speed             156
Country                156
Date                   156
Weather Description    156
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Weather Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
4,Bundaberg,AU,84.99,-24.8500,152.3500,broken clouds,
6,Hilo,US,75.99,19.7297,-155.0900,moderate rain,
11,Namatanai,PG,84.00,-3.6667,152.4333,broken clouds,
13,Hithadhoo,MV,84.09,-0.6000,73.0833,overcast clouds,
19,Gisborne,NZ,75.00,-38.6533,178.0042,clear sky,
23,Georgetown,MY,78.80,5.4112,100.3354,few clouds,
24,Pilar,BR,78.80,-9.5972,-35.9567,clear sky,
27,Saint-Philippe,RE,78.80,-21.3585,55.7679,few clouds,
29,Rikitea,PF,80.87,-23.1203,-134.9692,moderate rain,
33,Cabo San Lucas,MX,75.00,22.8909,-109.9124,few clouds,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # 6e. Make request and retrieve the JSON data from the search. 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
    
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()

In [18]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="Hotel_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))